<a href="https://colab.research.google.com/github/CaitLittlef/fws_sagebrush/blob/main/sei_trend_scratch_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Python/GEE set-up**


In [ ]:
## Initializing

# Installs geemap package
import subprocess

try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

#  Alt:
# !pip install geemap   

# Import packages
import ee
import folium
import geemap
import os
import numpy as np
from tabulate import tabulate
from folium import plugins
from random import seed
from random import randint
from pprint import pprint # tidy printing
from datetime import datetime
from ee.data import getInfo
import pickle
import statistics as st

from ee.data import getInfo
# If all you need to do is find out what's in the container, then just print() and inspect the result in the console.
# If, for some reason, you need to use Python running in the client to manipulate whatever is in the container, then use getInfo() to get the contents of the container and assign it to a variable
# Ref: https://developers.google.com/earth-engine/guides/client_server


# Trigger GEE authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

In [ ]:
# Mount google drive (or click in left panel)
from google.colab import drive
drive.mount('/content/drive')

# Set output directory
out_dir = '/content/drive/MyDrive/2FWS Sagebrush/FWS Sagebrush/analyses/output/GEE'
output = os.path.join(out_dir, 'temp.csv')  
print(out_dir)
print(output)

# Grab today
today = datetime.today().strftime('%Y-%m-%d')
print(today)

# **Data set-up**

In [3]:
## Calc set-up

# Set scale for calculations
scale = 90 #meters

# Get pixel area image in hectares for conversion from native units
sqm_per_ha = 1e4
sqm_per_acre = 4046.86
ha_area = ee.Image.pixelArea().divide(sqm_per_ha)
acre_area = ee.Image.pixelArea().divide(sqm_per_acre)

In [6]:
## COLLECT DATA

# Load shapefile with 3 groups: Great Basin, Great Plains, Intermountain West
eco = ee.FeatureCollection('projects/GEE_CSP/fws-sagebrush/eco_grps')
gb = eco.filter("group == 'Great Basin'")
gp = eco.filter("group == 'Great Plains'")
iw = eco.filter("group == 'Intermountain West'")

# Turn on/off masks based on ecoregions. High threshold for annual grass diff for Great Plains.
# ecoMask = eco #all
# ecoMask = gb ; mod = 8 ; high = 15
ecoMask = gp ; mod = 15 ; high = 42
# ecoMask = iw ; mod = 8 ; high = 15


# Select cores. Clip to desired AOI. Mask here isn't necessary (tho it is in wall-to-wall stressors below).Turn on/off cores by ecoregion.
cores_01 = ee.Image('users/DavidTheobald8/WAFWA/v11/SEIv11_1998_2001_90_20211228').select('Q5sc3')#.clip(ecoMask)
cores_06 = ee.Image('users/DavidTheobald8/WAFWA/v11/SEIv11_2003_2006_90_20211228').select('Q5sc3')#.clip(ecoMask)
cores_11 = ee.Image('users/DavidTheobald8/WAFWA/v11/SEIv11_2008_2011_90_20211228').select('Q5sc3')#.clip(ecoMask)
cores_16 = ee.Image('users/DavidTheobald8/WAFWA/v11/SEIv11_2013_2016_90_20211228').select('Q5sc3')#.clip(ecoMask)
cores_20 = ee.Image('users/DavidTheobald8/WAFWA/v11/SEIv11_2017_2020_90_20211228').select('Q5sc3')#.clip(ecoMask)

# # Turn into a collection
# coresList = [cores_01, cores_06, cores_11, cores_16, cores_20]
# print(coresList)
# len(coresList)

# coresColl = ee.ImageCollection(coresList)
# pprint(coresColl.getInfo())


# Select mean annual grass cover (don’t use Q3raw – not actually percent cover!). Mask to only those areas within range mask, else all western US.
ag_01 = ee.Image('users/DavidTheobald8/WAFWA/v11/SEIv11_1998_2001_90_20211228').select('annualG560m').clip(ecoMask).updateMask(cores_01.gte(1))
ag_06 = ee.Image('users/DavidTheobald8/WAFWA/v11/SEIv11_2003_2006_90_20211228').select('annualG560m').clip(ecoMask).updateMask(cores_01.gte(1))
ag_11 = ee.Image('users/DavidTheobald8/WAFWA/v11/SEIv11_2008_2011_90_20211228').select('annualG560m').clip(ecoMask).updateMask(cores_01.gte(1))
ag_16 = ee.Image('users/DavidTheobald8/WAFWA/v11/SEIv11_2013_2016_90_20211228').select('annualG560m').clip(ecoMask).updateMask(cores_01.gte(1))
ag_20 = ee.Image('users/DavidTheobald8/WAFWA/v11/SEIv11_2017_2020_90_20211228').select('annualG560m').clip(ecoMask).updateMask(cores_01.gte(1))


# Select mean tree cover
tree_01 = ee.Image('users/DavidTheobald8/WAFWA/v11/SEIv11_1998_2001_90_20211228').select('tree560m').clip(ecoMask).updateMask(cores_01.gte(1))
tree_06 = ee.Image('users/DavidTheobald8/WAFWA/v11/SEIv11_2003_2006_90_20211228').select('tree560m').clip(ecoMask).updateMask(cores_01.gte(1))
tree_11 = ee.Image('users/DavidTheobald8/WAFWA/v11/SEIv11_2008_2011_90_20211228').select('tree560m').clip(ecoMask).updateMask(cores_01.gte(1))
tree_16 = ee.Image('users/DavidTheobald8/WAFWA/v11/SEIv11_2013_2016_90_20211228').select('tree560m').clip(ecoMask).updateMask(cores_01.gte(1))
tree_20 = ee.Image('users/DavidTheobald8/WAFWA/v11/SEIv11_2017_2020_90_20211228').select('tree560m').clip(ecoMask).updateMask(cores_01.gte(1))

# Select HMI
hmi_01 = ee.Image('users/DavidTheobald8/WAFWA/v11/SEIv11_1998_2001_90_20211228').select('H560m').clip(ecoMask).updateMask(cores_01.gte(1))
hmi_06 = ee.Image('users/DavidTheobald8/WAFWA/v11/SEIv11_2003_2006_90_20211228').select('H560m').clip(ecoMask).updateMask(cores_01.gte(1))
hmi_11 = ee.Image('users/DavidTheobald8/WAFWA/v11/SEIv11_2008_2011_90_20211228').select('H560m').clip(ecoMask).updateMask(cores_01.gte(1))
hmi_16 = ee.Image('users/DavidTheobald8/WAFWA/v11/SEIv11_2013_2016_90_20211228').select('H560m').clip(ecoMask).updateMask(cores_01.gte(1))
hmi_20 = ee.Image('users/DavidTheobald8/WAFWA/v11/SEIv11_2017_2020_90_20211228').select('H560m').clip(ecoMask).updateMask(cores_01.gte(1))



# **Core areas trend analysis**

In [ ]:
# ####################
# #### CORE AREAS ####
# ####################

# def getArea(img, area_type, geom):
#     areas = img.pixelArea().divide(area_type).addBands(img).reduceRegion(reducer = ee.Reducer.sum().group(groupField = 1),
#                                                                          geometry = eco.geometry(), # maybe loop thru each ecoregion"?
#                                                                          scale = 90,
#                                                                          maxPixels = 1e11)
#     return areas
    
# boo = getArea(cores_01, sqm_per_acre, eco.geometry())
# pprint(boo.getInfo())


# # foo = getArea(coresColl, sqm_per_acre, eco.geometry()) # ERROR: 
# # AttributeError: 'ImageCollection' object has no attribute 'pixelArea'
# # Instead probably have to map function over image collection?


In [ ]:
####################
#### CORE AREAS ####
####################

# Run numbers for all years.
# FIXME: Do for each of the three ecoregions; turn this into a loop and generate tidy table outputs.

# Create new pixel area layer, divide that by conversion factor to convert to acres then tack cores back on for grouping. Gives pixel area in acres
# The groupField is the index of the band containing the zones by which to group the output. The first band is index 0, the second is index 1, etc.
area_cores_01 = cores_01.pixelArea().divide(sqm_per_acre).addBands(cores_01).reduceRegion(reducer = ee.Reducer.sum().group(groupField = 1), # could do count, sum, mean, etc.
                       geometry = eco.geometry(), 
                       scale = 90,  # Not necessary with counts, but necessary with aggregation (e.g., sum) b/c aggregation at 1-degree scale is usually not desired/intended
                       maxPixels = 1e11)  # To avoiding hitting max pixel value, either set this high, set scale to lower res, set bestEffort to True, which automatically computes a new (larger) scale such that maxPixels is not exceeded

area_cores_06 = cores_06.pixelArea().divide(sqm_per_acre).addBands(cores_06).reduceRegion(reducer = ee.Reducer.sum().group(groupField = 1), 
                       geometry = eco.geometry(), 
                       scale = 90,
                       maxPixels = 1e11)

area_cores_11 = cores_11.pixelArea().divide(sqm_per_acre).addBands(cores_11).reduceRegion(reducer = ee.Reducer.sum().group(groupField = 1), 
                       geometry = eco.geometry(), 
                       scale = 90,
                       maxPixels = 1e11)

area_cores_16 = cores_16.pixelArea().divide(sqm_per_acre).addBands(cores_16).reduceRegion(reducer = ee.Reducer.sum().group(groupField = 1), 
                       geometry = eco.geometry(), 
                       scale = 90,
                       maxPixels = 1e11)

area_cores_20 = cores_20.pixelArea().divide(sqm_per_acre).addBands(cores_20).reduceRegion(reducer = ee.Reducer.sum().group(groupField = 1), 
                       geometry = eco.geometry(), 
                       scale = 90,
                       maxPixels = 1e11)


print(ecoMask)
pprint(area_cores_01.getInfo()) # Results above are dictionaries (nested JSON format); "pretty" print them.
pprint(area_cores_06.getInfo())
pprint(area_cores_11.getInfo())
pprint(area_cores_16.getInfo())
pprint(area_cores_20.getInfo())



## FYI
## By default, reducers applied to imagery weight the inputs according to the mask value.
## For example, if a clipping cuts a pixel in half, GEE would default to only letting that pixel contribute 1/2.
## Unweighting would force that pixel to contribute equally.

## Could combine reducers -- here, mean and standard deviation reducers. Then set reducer = reducers.
# reducers = ee.Reducer.mean().combine(
#   reducer2=ee.Reducer.stdDev(),
#   sharedInputs=True
# )

## Refs:
## GEE calc are (in JavaScript): https://spatialthoughts.com/2020/06/19/calculating-area-gee/
## Justin's CAP reference: https://github.com/csp-inc/cap-30x30/blob/main/Notebooks/scenarios-comparison.ipynb
## General reducer ref: https://colab.research.google.com/github/csaybar/EEwPython/blob/master/5_Reducer.ipynb#scrollTo=ta9risk8NRSm ; https://github.com/csaybar/EEwPython/blob/master/5_Reducer.ipynb
## GEEMAP: https://geemap.org/tutorials/

# **Stressors trend analysis**

In [ ]:
######################
#### ANNUAL GRASS ####
######################

# Need to mask these by sagebrush extent (just use core area); then divide by different ecoregions.

# Bin stressor by break-points identified in https://docs.google.com/document/d/1swlSAlQ6TUAES-0pk4jklBW34ywWvhgCIteox13OJO0/edit#
# “no to low” (0-8% cover), 
# moderate (>8-15%), 
# high (>15%) ; except for Great Plains (>42) (set above at ecoregion selection)



# pprint(ag_01.getInfo())
bins = ag_01.gte(0).add(ag_01.gt(mod)).add(ag_01.gt(high)) # 
area_ag_01 = ag_01.pixelArea().divide(sqm_per_acre).addBands(bins).reduceRegion(reducer = ee.Reducer.sum().group(groupField = 1), 
                       geometry = eco.geometry(), 
                       scale = 90,
                       maxPixels = 1e11)

bins = ag_06.gte(0).add(ag_06.gt(mod)).add(ag_06.gt(high))
area_ag_06 = ag_06.pixelArea().divide(sqm_per_acre).addBands(bins).reduceRegion(reducer = ee.Reducer.sum().group(groupField = 1), 
                       geometry = eco.geometry(), 
                       scale = 90,
                       maxPixels = 1e11)

bins = ag_11.gte(0).add(ag_11.gt(mod)).add(ag_11.gt(high))
area_ag_11 = ag_11.pixelArea().divide(sqm_per_acre).addBands(bins).reduceRegion(reducer = ee.Reducer.sum().group(groupField = 1), 
                       geometry = eco.geometry(), 
                       scale = 90,
                       maxPixels = 1e11)

bins = ag_16.gte(0).add(ag_16.gt(mod)).add(ag_16.gt(high))
area_ag_16 = ag_16.pixelArea().divide(sqm_per_acre).addBands(bins).reduceRegion(reducer = ee.Reducer.sum().group(groupField = 1), 
                       geometry = eco.geometry(), 
                       scale = 90,
                       maxPixels = 1e11)

bins = ag_20.gte(0).add(ag_20.gt(mod)).add(ag_20.gt(high))
area_ag_20 = ag_20.pixelArea().divide(sqm_per_acre).addBands(bins).reduceRegion(reducer = ee.Reducer.sum().group(groupField = 1), 
                       geometry = eco.geometry(), 
                       scale = 90,
                       maxPixels = 1e11)

print(ecoMask)
print('Mod percent cover threshold:', mod)
print('High percent cover threshold:', high)
pprint(area_ag_01.getInfo())
pprint(area_ag_06.getInfo())
pprint(area_ag_11.getInfo())
pprint(area_ag_16.getInfo())
pprint(area_ag_20.getInfo())

# https://developers.google.com/earth-engine/guides/image_relational#colab-python
# https://courses.spatialthoughts.com/end-to-end-gee.html#calculating-area-by-class



In [ ]:
#####################################
#### ANNUAL GRASS THREAT BY CORE ####
#####################################

# Nb Adding bins after updating mask seems to have removed mask; set mask after all datasets are squared away.

bins = ag_20.gte(0).add(ag_20.gt(mod)).add(ag_20.gt(high))
area_ag_cores_20_1 = cores_20.pixelArea().divide(sqm_per_acre).addBands(bins).updateMask(cores_20.eq(1)).reduceRegion(reducer = ee.Reducer.sum().group(groupField = 1), 
                       geometry = eco.geometry(), 
                       scale = 90,
                       maxPixels = 1e11)

area_ag_cores_20_2 = cores_20.pixelArea().divide(sqm_per_acre).addBands(bins).updateMask(cores_20.eq(2)).reduceRegion(reducer = ee.Reducer.sum().group(groupField = 1), 
                       geometry = eco.geometry(), 
                       scale = 90,
                       maxPixels = 1e11)

area_ag_cores_20_3 = cores_20.pixelArea().divide(sqm_per_acre).addBands(bins).updateMask(cores_20.eq(3)).reduceRegion(reducer = ee.Reducer.sum().group(groupField = 1), 
                       geometry = eco.geometry(), 
                       scale = 90,
                       maxPixels = 1e11)

print(ecoMask)
print('Mod percent cover threshold:', mod)
print('High percent cover threshold:', high)
pprint(area_ag_cores_20_1.getInfo())
pprint(area_ag_cores_20_2.getInfo())
pprint(area_ag_cores_20_3.getInfo())

In [ ]:
##############
#### TREE ####
##############

# Bin stressor by break-points identified in https://docs.google.com/document/d/1swlSAlQ6TUAES-0pk4jklBW34ywWvhgCIteox13OJO0/edit#
# “no to low” (0-2% tree cover), 
# moderate (>2-10%), 
# high (>10-20%), and 
# very high (>20%)


bins = tree_01.gte(0.0).add(tree_01.gt(2)).add(tree_01.gt(10)).add(tree_01.gt(20))
area_tree_01 = tree_01.pixelArea().divide(sqm_per_acre).addBands(bins).reduceRegion(reducer = ee.Reducer.sum().group(groupField = 1), 
                       geometry = eco.geometry(), 
                       scale = 90,
                       maxPixels = 1e11)

bins = tree_06.gte(0.0).add(tree_06.gt(2)).add(tree_06.gt(10)).add(tree_06.gt(20))
area_tree_06 = tree_06.pixelArea().divide(sqm_per_acre).addBands(bins).reduceRegion(reducer = ee.Reducer.sum().group(groupField = 1), 
                       geometry = eco.geometry(), 
                       scale = 90,
                       maxPixels = 1e11)

bins = tree_11.gte(0.0).add(tree_11.gt(2)).add(tree_11.gt(10)).add(tree_11.gt(20))
area_tree_11 = tree_11.pixelArea().divide(sqm_per_acre).addBands(bins).reduceRegion(reducer = ee.Reducer.sum().group(groupField = 1), 
                       geometry = eco.geometry(), 
                       scale = 90,
                       maxPixels = 1e11)

bins = tree_16.gte(0.0).add(tree_16.gt(2)).add(tree_16.gt(10)).add(tree_16.gt(20))
area_tree_16 = tree_16.pixelArea().divide(sqm_per_acre).addBands(bins).reduceRegion(reducer = ee.Reducer.sum().group(groupField = 1), 
                       geometry = eco.geometry(), 
                       scale = 90,
                       maxPixels = 1e11)

bins = tree_20.gte(0.0).add(tree_20.gt(2)).add(tree_20.gt(10)).add(tree_20.gt(20))
area_tree_20 = tree_20.pixelArea().divide(sqm_per_acre).addBands(bins).reduceRegion(reducer = ee.Reducer.sum().group(groupField = 1), 
                       geometry = eco.geometry(), 
                       scale = 90,
                       maxPixels = 1e11)
print(ecoMask)
pprint(area_tree_01.getInfo())
pprint(area_tree_06.getInfo())
pprint(area_tree_11.getInfo())
pprint(area_tree_16.getInfo())
pprint(area_tree_20.getInfo())


In [ ]:
#############################
#### TREE THREAT BY CORE ####
#############################

# Nb Adding bins after updating mask seems to have removed mask; set mask after all datasets are squared away.

bins = tree_20.gte(0.0).add(tree_20.gt(2)).add(tree_20.gt(10)).add(tree_20.gt(20))
area_tree_cores_20_1 = cores_20.pixelArea().divide(sqm_per_acre).addBands(bins).updateMask(cores_20.eq(1)).reduceRegion(reducer = ee.Reducer.sum().group(groupField = 1), 
                       geometry = eco.geometry(), 
                       scale = 90,
                       maxPixels = 1e11)

area_tree_cores_20_2 = cores_20.pixelArea().divide(sqm_per_acre).addBands(bins).updateMask(cores_20.eq(2)).reduceRegion(reducer = ee.Reducer.sum().group(groupField = 1), 
                       geometry = eco.geometry(), 
                       scale = 90,
                       maxPixels = 1e11)

area_tree_cores_20_3 = cores_20.pixelArea().divide(sqm_per_acre).addBands(bins).updateMask(cores_20.eq(3)).reduceRegion(reducer = ee.Reducer.sum().group(groupField = 1), 
                       geometry = eco.geometry(), 
                       scale = 90,
                       maxPixels = 1e11)

print(ecoMask)
# print('High percent cover threshold:', high)
pprint(area_tree_cores_20_1.getInfo())
pprint(area_tree_cores_20_2.getInfo())
pprint(area_tree_cores_20_3.getInfo())

In [ ]:
#############
#### HMI ####
#############


# Bin stressor not based on methods above but given in this spreadsheet https://docs.google.com/spreadsheets/d/15Q9FtCgVd_fMYIQJHaL8vgkjbkSa7sg9vonE2WJ7oVw/edit#gid=342109086
# “no to low” (0-3), 
# moderate (>3-15), 
# high (>15%)


bins = hmi_01.gte(0).add(hmi_01.gt(3)).add(hmi_01.gt(15))
area_hmi_01 = hmi_01.pixelArea().divide(sqm_per_acre).addBands(bins).reduceRegion(reducer = ee.Reducer.sum().group(groupField = 1), 
                       geometry = eco.geometry(), 
                       scale = 90,
                       maxPixels = 1e11)

bins = hmi_06.gte(0).add(hmi_06.gt(3)).add(hmi_06.gt(15))
area_hmi_06 = hmi_06.pixelArea().divide(sqm_per_acre).addBands(bins).reduceRegion(reducer = ee.Reducer.sum().group(groupField = 1), 
                       geometry = eco.geometry(), 
                       scale = 90,
                       maxPixels = 1e11)

bins = hmi_11.gte(0).add(hmi_11.gt(3)).add(hmi_11.gt(15))
area_hmi_11 = hmi_11.pixelArea().divide(sqm_per_acre).addBands(bins).reduceRegion(reducer = ee.Reducer.sum().group(groupField = 1), 
                       geometry = eco.geometry(), 
                       scale = 90,
                       maxPixels = 1e11)

bins = hmi_16.gte(0).add(hmi_16.gt(3)).add(hmi_16.gt(15))
area_hmi_16 = hmi_16.pixelArea().divide(sqm_per_acre).addBands(bins).reduceRegion(reducer = ee.Reducer.sum().group(groupField = 1), 
                       geometry = eco.geometry(), 
                       scale = 90,
                       maxPixels = 1e11)

bins = hmi_20.gte(0).add(hmi_20.gt(3)).add(hmi_20.gt(15))
area_hmi_20 = hmi_20.pixelArea().divide(sqm_per_acre).addBands(bins).reduceRegion(reducer = ee.Reducer.sum().group(groupField = 1), 
                       geometry = eco.geometry(), 
                       scale = 90,
                       maxPixels = 1e11)

print(ecoMask)
pprint(area_hmi_01.getInfo())
pprint(area_hmi_06.getInfo())
pprint(area_hmi_11.getInfo())
pprint(area_hmi_16.getInfo())
pprint(area_hmi_20.getInfo())



In [ ]:
############################
#### HMI THREAT BY CORE ####
############################

# Nb Adding bins after updating mask seems to have removed mask; set mask after all datasets are squared away.

bins = hmi_20.gte(0).add(hmi_20.gt(3)).add(hmi_20.gt(15))
area_hmi_cores_20_1 = cores_20.pixelArea().divide(sqm_per_acre).addBands(bins).updateMask(cores_20.eq(1)).reduceRegion(reducer = ee.Reducer.sum().group(groupField = 1), 
                       geometry = eco.geometry(), 
                       scale = 90,
                       maxPixels = 1e11)

area_hmi_cores_20_2 = cores_20.pixelArea().divide(sqm_per_acre).addBands(bins).updateMask(cores_20.eq(2)).reduceRegion(reducer = ee.Reducer.sum().group(groupField = 1), 
                       geometry = eco.geometry(), 
                       scale = 90,
                       maxPixels = 1e11)

area_hmi_cores_20_3 = cores_20.pixelArea().divide(sqm_per_acre).addBands(bins).updateMask(cores_20.eq(3)).reduceRegion(reducer = ee.Reducer.sum().group(groupField = 1), 
                       geometry = eco.geometry(), 
                       scale = 90,
                       maxPixels = 1e11)

print(ecoMask)
# print('High percent cover threshold:', high)
pprint(area_hmi_cores_20_1.getInfo())
pprint(area_hmi_cores_20_2.getInfo())
pprint(area_hmi_cores_20_3.getInfo())

# **Visualization**
With quick GEE glimpse then folium

In [ ]:
# GEE
Map = geemap.Map(center=[40,-110], zoom=5)

# Set visualization parameters.
visCores = {
  'min': 1,
  'max': 3,
  'palette': ['052963', '7fa2db', 'f0ce7a']}

visPercCov = {
  'min':0,
  'max':42,
  'palette': ['black', 'yellow']
}

visQbin = {
  'min':1,
  'max':3,
  'palette': ['green', 'yellow', 'red']
}

 # Add Earth Engine layers to Map
Map.addLayer(gb, {'color':'grey'}, name = "Great Basin", shown = True, opacity = 0.8)
Map.addLayer(gp, {'color':'green'}, name = "Great Plains", shown = True, opacity = 0.8)
Map.addLayer(iw, {'color':'purple'}, name = "Intermountain West", shown = True, opacity = 0.8)

# Map.addLayer(cores_20_1, {'color':'blue'}, name = "CSA Cores 2020", shown = True, opacity = 0.8)
# Map.addLayer(cores_20_2, {'color':'green'}, name = "GOA Cores 2020", shown = True, opacity = 0.8)
# Map.addLayer(cores_20_3, {'color':'yellow'}, name = "HIA Cores 2020", shown = True, opacity = 0.8)

# Map.addLayer(cores_01, visCores, '2001 cores', True, 1)
# Map.addLayer(cores_11, visCores, '2011 cores', True, 1)
# Map.addLayer(cores_20, visCores, '2020 cores', True, 1)
# Map.addLayer(ag_20, visPercCov, 'ag 2020', True, 1)

# temp = ee.Image('users/DavidTheobald8/WAFWA/v11/SEIv11_2017_2020_90_20211228').select('annualG560m').updateMask(cores_01.gte(1))

# Map.addLayer(ag_11, visPercCov, 'ag 2011', True, 1)
# Map.addLayer(ag_16, visPercCov, 'ag 2016', True, 1)
# Map.addLayer(ag_20, visPercCov, 'ag 2020', True, 1)
# Map.addLayer(temp, visPercCov, 'ag 2020 all', True, 1)


# bins = ag_20.gte(0).add(ag_20.gt(8)).add(ag_20.gt(42))
# Map.addLayer(bins, visQbin, 'Ag bins', True, 1)


Map

In [ ]:
 # Establish folium settings

# Get other foliumbasemaps
basemaps = {
        'Esri Satellite': folium.TileLayer(
            tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
            attr = 'Esri',
            name = 'Esri Satellite',
            overlay = True,
            control = True
        ),
    'Carto Dark': folium.TileLayer('cartodbdark_matter'),
    'Carto Positron': folium.TileLayer('cartodbpositron')
    }

# Define a method for displaying Earth Engine image tiles on a folium map.
def add_ee_layer(self, ee_object, vis_params, name):
    
    try:    
        # display ee.Image()
        if isinstance(ee_object, ee.image.Image):    
            map_id_dict = ee.Image(ee_object).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.ImageCollection()
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):    
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.Geometry()
        elif isinstance(ee_object, ee.geometry.Geometry):    
            folium.GeoJson(
            data = ee_object.getInfo(),
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
        # display ee.FeatureCollection()
        elif isinstance(ee_object, ee.featurecollection.FeatureCollection):  
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
    
    except:
        print("Could not display {}".format(name))
        
# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [ ]:
# folium.Map(['47.38','-119.495'])
# visF = {'min': 0, 'max': 1, 'palette': ["000004","180F3E", "451077", "721F81", "9F2F7F","CD4071","F1605D","FD9567","FEC98D","FCFDBF"]}
# visP = {'min': 0, 'max': 700, 'palette': ["000004","180F3E", "451077", "721F81", "9F2F7F","CD4071","F1605D","FD9567","FEC98D","FCFDBF"]}
# visP1 = {'palette':'#48d6fa'}
# visP2 = {'palette':'red'}
# visP3 = {'palette':'magenta'}
# visP4 = {'palette':'9F2F7F'}
# visP5 = {'palette':'FEC98D'}
visCores = {
  'min': 1,
  'max': 3,
  'palette': ['052963', '7fa2db', 'f0ce7a']}

# Create a folium map object.
# my_map = folium.Map(location=['40','-110'], zoom_start=5, height=500)

my_map = folium.Map(location = [40,-100], zoom_start = 5)

# # Add custom basemaps
basemaps['Carto Positron'].add_to(my_map)
basemaps['Carto Dark'].add_to(my_map)

# Add Layers
# my_map.add_ee_layer(rand_ex.randomVisualizer(), {},'rand')
my_map.add_ee_layer(cores_01, visCores, 'cores 2001')
# my_map.add_ee_layer(expImg, visP1, 'burn')
# my_map.add_ee_layer(burn_prob, visF, 'burn')
# my_map.add_ee_layer(pa_mask, {'palette':'white'},'PA')


# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Add fullscreen button
plugins.Fullscreen().add_to(my_map)

# Display the map.
display(my_map)